In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './국토교통부_실거래가_공개시스템/'
filenames = [f for f in os.listdir(basedir) if (f.startswith('아파트(매매)')) & (f.endswith('.csv'))]
filenames

['아파트(매매)__실거래가_20201211150806.csv',
 '아파트(매매)__실거래가_20201211152927.csv',
 '아파트(매매)__실거래가_20201214122854.csv',
 '아파트(매매)__실거래가_20201214124757.csv',
 '아파트(매매)__실거래가_20201214133432.csv',
 '아파트(매매)__실거래가_20201214141339.csv',
 '아파트(매매)__실거래가_20201214145855.csv']

In [3]:
prac_df = pd.read_csv(basedir + filenames[0], encoding='euc-kr', header=15)
print(prac_df.shape)
prac_df.head()

(68136, 12)


,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,"57,000",2,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,"57,000",2,1988,언주로 103
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,"55,000",5,1987,언주로 3
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,"58,250",4,1987,언주로 3
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,"60,000",5,1987,언주로 3


In [5]:
prac_df['단지명'].nunique()

4500

In [6]:
prac_df['단지명'].unique()[:100]

array(['개포2차현대아파트(220)', '개포6차우성아파트1동~8동', '개포우성3차', '개포자이(12-2)',
       '개포주공 1단지', '개포주공 3단지', '개포주공 4단지', '개포주공 5단지', '개포주공 6단지',
       '개포주공 7단지', '개포현대200동', '경남1', '경남2차', '뉴현대파크빌', '리더스', '삼익대청아파트',
       '새롬(1164-13)', '성원대치2단지아파트', '시영', '아람손프라자/비바빌', '우성8',
       '우성9차아파트 901동, 902동', '우찬현대(1260-11)', '우찬현대(1260-4)', '주공2',
       '포이벨리', '현대1차101동~106동', '현대2', '현대3', '거평프리젠', '경남논현',
       '논현동월드메르디앙', '논현동한화꿈에그린', '논현라폴리움', '논현베르빌', '논현파라곤', '논현한가람빌라트',
       '동부센트레빌', '동양파라곤', '동현아파트 1~6', '두산위브1단지', '두산위브2단지', '라이헤스빌',
       '리더스빌', '마일스디오빌', '벨러스빌', '삼성빌라트', '세호시티라이프', '스위트캐슬레지던스',
       '신동아(22)', '쌍용', '우민', '우성', '제이빌', '지고하임(101동)', '지고하임(102동)',
       '청학아파트 에이동,비동,씨동', '테크노빌라트', '프라임', '학동', '한양빌라트', '한양수자인어반게이트',
       '한진로즈힐', '한화꿈에그린2차', '현대넥서스 B동', '현대인텔렉스', '힐타워', '개포우성1', '개포우성2',
       '국제', '노빌리티빌리지', '대우아이빌멤버스(891-26)', '대우아이빌명문가(891-23)',
       '대치동우정에쉐르1', '대치삼성', '대치아이파크', '대치우성아파트1동, 2동, 3동, 5동, 6동, 7동',
       '대치주성', '대치하나빌', '대치한신휴플러스',

In [6]:
%%time
main_data_df = pd.read_csv('./prepped_data/main_data_ver_2.3.csv')

1


c:\users\max6296\ml\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


2
3
4
5
Wall time: 23.1 s


In [10]:
main_data_df.head()

,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [11]:
main_data_df.drop(columns=['시군구', '번지', '본번', '부번', '건물명', '도로명', '시', '구', '동'], inplace=True)
main_data_df.head()

,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,가격/면적,year,month,target_log_transformed,target/area_log_transformed,건물종류,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,매매,77.75,201309,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,매매,77.75,201312,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,매매,67.28,201302,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,매매,67.28,201302,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,매매,67.28,201305,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [12]:
main_data_df['대장구분명'].value_counts()

일반       4553162
BL          3759
산           2091
가            508
지구BL         346
Name: 대장구분명, dtype: int64

In [13]:
main_data_df['대장구분코드'].value_counts()

1    4553162
5       3759
2       2091
3        508
7        346
Name: 대장구분코드, dtype: int64

In [14]:
main_data_df['대장구분명'] = main_data_df['대장구분명'].apply(lambda x: '산' if x == '산' else '일반')

In [15]:
main_data_df['대장구분명'].value_counts()

일반    4557775
산        2091
Name: 대장구분명, dtype: int64

In [16]:
main_data_df['대장구분코드'] = main_data_df['대장구분명'].apply(lambda x: 1 if x == '일반' else 2)
main_data_df['대장구분코드'].value_counts()

1    4557775
2       2091
Name: 대장구분코드, dtype: int64

In [17]:
main_data_df.head()

,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,가격/면적,year,month,target_log_transformed,target/area_log_transformed,건물종류,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,매매,77.75,201309,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,매매,77.75,201312,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,매매,67.28,201302,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,매매,67.28,201302,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,매매,67.28,201305,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [18]:
main_data_df.drop(columns=['전월세매매구분'], inplace=True)

In [19]:
main_data_df.head()

,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,가격/면적,year,month,target_log_transformed,target/area_log_transformed,건물종류,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,77.75,201309,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,77.75,201312,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,67.28,201302,11,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,67.28,201302,22,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,67.28,201305,10,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [21]:
main_data_df.drop(columns=['계약년월', '계약일'], inplace=True)
main_data_df.head()

,전용면적(㎡),금액(만원),월세(만원),층,건축년도,가격/면적,year,month,target_log_transformed,target/area_log_transformed,건물종류,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,77.75,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,77.75,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,67.28,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,67.28,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,67.28,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [22]:
main_data_df.rename(columns={'year':'년', 'month':'월'}, inplace=True)
main_data_df.head()

,전용면적(㎡),금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,77.75,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,77.75,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,67.28,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,67.28,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,67.28,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [23]:
main_data_df.drop(columns=['건물종류'], inplace=True)
main_data_df.head()

,전용면적(㎡),금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,trade_type,지번주소,대장구분코드,대장구분명,도로명주소
0,77.75,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
1,77.75,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0
2,67.28,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
3,67.28,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0
4,67.28,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0


In [24]:
main_data_df['건물나이'] = main_data_df['년'] - main_data_df['건축년도']

In [25]:
main_data_df.head()

,전용면적(㎡),금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,trade_type,지번주소,대장구분코드,대장구분명,도로명주소,건물나이
0,77.75,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0,25
1,77.75,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반,서울특별시 강남구 언주로 103-0,25
2,67.28,55000,0,5,1987,817.479191,2013,2,10.915088,6.706225,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0,26
3,67.28,58250,0,4,1987,865.784780,2013,2,10.972499,6.763636,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0,26
4,67.28,60000,0,5,1987,891.795482,2013,5,11.002100,6.793237,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반,서울특별시 강남구 언주로 3-0,26


In [26]:
main_data_df = main_data_df[['지번주소', '도로명주소', '대장구분명', '대장구분코드', 'trade_type', '전용면적(㎡)',
                             '금액(만원)', '월세(만원)', '가격/면적', '층', '년', '월', '건축년도', '건물나이',
                             'target_log_transformed', 'target/area_log_transformed'
                            ]]

In [27]:
main_data_df.head()

,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,9,1988,25,10.950807,6.597308
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,12,1988,25,10.950807,6.597308
2,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,55000,0,817.479191,5,2013,2,1987,26,10.915088,6.706225
3,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,58250,0,865.784780,4,2013,2,1987,26,10.972499,6.763636
4,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,60000,0,891.795482,5,2013,5,1987,26,11.002100,6.793237


In [28]:
main_data_df.to_csv('main_data_ver_2.4.csv', index=False)